<a href="https://colab.research.google.com/github/Fotouhif/Project_pointcloud_RNN/blob/Preprocessing_data/Preprocessing_data/ME570project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Don't forget to turn on GPU if you want to start training directly.**

Runtime -> Change runtime type-> Hardware accelerator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
%cd /content/drive/My Drive/PhD/courses/ME_570X/Project/Colab_pointnet

/content/drive/My Drive/PhD/courses/ME_570X/Project/Colab_pointnet


# **Import packages**

In [2]:
import numpy as np
import math
import random
import os
import torch
import scipy.spatial.distance
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import plotly.graph_objects as go
import plotly.express as px

In [3]:
!pip install path.py;
from path import Path

In [4]:
random.seed = 42

## **Download Dataset if you do not have it!**

In [ ]:
!wget http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip

In [ ]:
!unzip -q ModelNet10.zip;

# Classes Folders

In [6]:
path = Path("ModelNet10")

In [7]:
folders = [dir for dir in sorted(os.listdir(path)) if os.path.isdir(path/dir)]
classes = {folder: i for i, folder in enumerate(folders)};
classes

{'bathtub': 0,
 'bed': 1,
 'chair': 2,
 'desk': 3,
 'dresser': 4,
 'monitor': 5,
 'night_stand': 6,
 'sofa': 7,
 'table': 8,
 'toilet': 9}

This dataset consists of **.off** files that contain meshes represented by *vertices* and *triangular faces*. 

We will need a function to read this type of files:

In [8]:
def read_off(file):
    if 'OFF' != file.readline().strip():
        raise('Not a valid OFF header')
    n_verts, n_faces, __ = tuple([int(s) for s in file.readline().strip().split(' ')])
    verts = [[float(s) for s in file.readline().strip().split(' ')] for i_vert in range(n_verts)]
    faces = [[int(s) for s in file.readline().strip().split(' ')][1:] for i_face in range(n_faces)]
    return verts, faces

In [9]:
with open(path/"chair/train/chair_0010.off", 'r') as f:
  verts, faces = read_off(f)

In [10]:
i,j,k = np.array(faces).T
x,y,z = np.array(verts).T

In [11]:
len(x)

1842